<a href="https://colab.research.google.com/github/nicklausmillican/StatisticalRethinkingIISolutions/blob/main/StatisticalRethinkingSolutions2_Ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8

In [ ]:
install.packages(c("coda","mvtnorm","devtools","loo","dagitty"))
devtools::install_github("rmcelreath/rethinking@slim")

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘numDeriv’, ‘shiny’, ‘downlit’, ‘htmlwidgets’, ‘abind’, ‘tensorA’, ‘distributional’, ‘Rcpp’, ‘miniUI’, ‘pkgdown’, ‘profvis’, ‘urlchecker’, ‘checkmate’, ‘matrixStats’, ‘posterior’, ‘V8’





shape (NA -> 1.4.6.1) [CRAN]


Installing 1 packages: shape

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpAgTvmt/remotes1b55b214b8b/rmcelreath-rethinking-cbcb8ba/DESCRIPTION’ ... OK
* preparing ‘rethinking’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
* building ‘rethinking_2.13.2.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(rethinking)

Loading required package: parallel

rethinking (Version 2.13.2)


Attaching package: ‘rethinking’


The following object is masked from ‘package:stats’:

    rstudent




## Easy

### 8E1
#### Question
For each of the causal relationships below, name a hypothetical third variable that would lead to an interaction effect.  
> (1) Bread dough rises because of yeast.  

> (2) Education leads to higher income.  

> (3) Gasoline makes a car go.

#### Answers
> (1) (Had to google "How does yeast make bread rise")  Yeast gobbles up sugar in the dough, releasing gasses that are trapped in the matrix of the dough and causes rising.  This means that *sugar* is a third variable that would lead to an interaction; something like $rise = \alpha + ... + \beta_{y \times s} \times yeast \times sugar$.

> (2) Education may lead to higher income, depending on the area of specialization.  E.g., an investment banker will probably make more than a nurse.  So there is an interaction term like $income = \alpha + ... + \beta_{e \times s} \times education \times specialization$

> (3) Gas powers a car, contingent on the proper function of the rest of the car.  E.g., a car with no wheels will not move regardless of how much gas you give it.  $GO = \alpha + ... + \beta_{g \times w} \times gas \times functioning$


### 8E2-8E3
#### Question E2
Which of the following explanations invokes an interaction?  

> (1) Caramelizing onions requires cooking over low heat and making sure the onions do not dry out.  

> (2) A car will go faster when it has more cylinders or when it has a better fuel injector.  

> (3) Most people acquire their political beliefs from their parents, unless they get them instead from their friends.  

> (4) Intelligent animal species tend to be either highly social or have manipulative appendages (hands, tentacles, etc.).

#### Question E3
For each of the explanations in 8E2, write a linear model that expresses the stated relationship.

##### Answers
> (1) I don't know how carmelizing works, either.  Google tells me that it involves the non-enzymatic breakdown of sugar and the removal of water (which seems to contradict the question).  I think there's an interaction here since (I imagine, heat will not carmelize an onion if it's submerged in a pot of boiling water--so the effect of heat depends on moisture.  $$C = \beta_H H + \beta_M M + \beta_{HM} H M$$.  I don't have an intercept here because I imagine that no heat will produce no carmelizing.

> (2) I'm going to imagine that this could be an interactive effect: a car with more cylinders will perform better-than-linearly with more better fuel injectors.  In this case, $$S = \beta_C C + \beta_F F + \beta_{CF} C F$$.  But I could also imagine that any ole fuel injector will do the trick as long as it works, in which case, this is additive like $$S = \beta_C C + \beta_F F$$. Again, I don't have an intercept because a car with no cylinders probably won't go very fast...

> (3) This one is tricky.  It's describing an either-or situation.  Either a person gets their political beliefs from their parents or from their friends.  But this is NOT an interaction, as we've learned so far, since the effect of parents does not depend on the effect of friends.  We will later learn that the one's political beliefs may not be a linear function, and that this will cause interaction between parents and friends...just not interaction in the way we've learned so far.  So I'm going to say a linear model is like $$P = \alpha + \beta_I I$$, where $I$ is an indicator variable representing the source of influence (parents or friends).

> (4) No interaction here: the effect of sociality is not be described as being modulated by the effect of appendages (and vice versa).  $$I = \alpha + \beta_S S + \beta_A A$$

## Medium

### 8M1
Recall the tulips example from the chapter. Suppose another set of treatments adjusted the temperature in the greenhouse over two levels: cold and hot. The data in the chapter were collected at the cold temperature. You find none of the plants grown under the hot temperature developed any blooms at all, regardless of the water and shade levels. Can you explain this result in terms of interactions between water, shade, and temperature?

#### Answer
In the chapter, the final model used was

$$\mu_i = \alpha + \beta_W w_i + \beta_S s_i + \beta_{WS} w_i s_i$$

Now we're asked about another explanatory variable *heat*.  We find that *heat* has completely prevented any tulips in our sample to grow.  So apparently, there is an interaction of *water* and *shade* with *heat*: the effect of *water*, *shade*, and the interaction of *water and shade* depends on *heat*.

### 8M2
Can you invent a regression equation that would make the bloom size zero, whenever the temperature is hot?

#### Answer
We might think that a regression would look like

$$\mu_i = \alpha + \beta_W w_i + \beta_S s_i + \beta_H h_i +
\beta_{WS} w_i s_i + \beta_{WH} w_i h_i + \beta_{SH} s_i h_i +
\beta_{WSH} w_i s_i h_i$$

but our results suggest something different.  In this formula, *heat* might zero-out any term that include $h$, but the means that terms $\alpha + \beta_W w_i + \beta_S s_i + \beta_{WS} w_i s_i$ could lead to non-zero blooms.  Apparently, *heat* interacts with these too.  So we might have the following regression instead:

$$\mu_i = (1 - h_i) (\alpha + \beta_W w_i + \beta_S s_i + \beta_{WS} w_i s_i)$$

Now, when *heat* is applicable, $(1 - h_i) = 0$ such that $\mu_i = 0$ too.

### 8M3
In parts of North America, ravens depend upon wolves for their food. This is because ravens are carnivorous but cannot usually kill or open carcasses of prey. Wolves however can and do kill and tear open animals, and they tolerate ravens co-feeding at their kills. This species relationship is generally described as a “species interaction.” Can you invent a hypothetical set of data on raven population size in which this relationship would manifest as a statistical interaction? Do you think the biological interaction could be linear? Why or why not?

#### Answer
The way "species interaction" is described, it is NOT a *statistical* interaction.  The description of a species interaction is really describing a *statistical dependency*, where $P_{wolf}$ is an explanatory variable in a regression representing the outcome $P_{raven}$.  For example, the question could be describing a regression like:

$$P_{raven} = \alpha + \beta_W P_{wolf} + ...$$

In order for there to be a statistical interaction between $P_{raven}$ and $P_{wolf}$, $P_{raven}$ would have to enter the right-hand side of the regression--$P_{raven}$ would depend on itself (this can be modeled with differential equations, which the book will go with later chapters).

Nevertheless, there may be (statistical) interaction terms in the regression for $P_{raven}$ that do not include $P_{raven}$ as an explanatory variable.  For example, the populations of small prey (which the ravens may be able to hunt themselves, without relying on the prowess of wolves) and the large prey (which the ravens can take advantage of via the efforts of wolves).  Now we might have several interaction terms.

$$P_{raven} = \alpha +
\beta_W P_{wolf} + \beta_S P_{sm.prey} + \beta_L P_{lg.prey} +
\beta_{WS} P_{wolf} P_{sm.prey} + \beta_{WL} P_{wolf} P_{lg.prey} + \beta_{SL} P_{sm.prey} P_{lg.prey} +
\beta_{WSL} P_{wolf} P_{sm.prey} P_{lg.prey}$$

Some of these terms may not make sense in ecological reality.  For example, $\beta_{WS} P_{wolf} P_{sm.prey} = \beta_{SL} P_{sm.prey} P_{lg.prey} = 0$ if wolves don't hunt small prey and prey of different sizes don't affect the availability of either sized prey.  So we might have:

$$P_{raven} = \alpha +
\beta_W P_{wolf} + \beta_S P_{sm.prey} + \beta_L P_{lg.prey}
+ \beta_{WL} P_{wolf} P_{lg.prey} +
\beta_{WSL} P_{wolf} P_{sm.prey} P_{lg.prey}$$

So let's simulate some data here.

In [34]:
n <- 1000
pWolf <- round(runif(n=n, min=0, max=100), 0)
pSm <- round(runif(n=n, min=0, max=100), 0)
pLg <- round(pmax(runif(n=n, min=0, max=100) - 0.1*pWolf, 0), 0)
pRaven <- round(50 + 0*pWolf + 0.0001*pSm + 0.0002*pWolf*pLg + 0.0003*pWolf*pSm*pLg, 0)

d <- data.frame(pWolf=pWolf, pSm=pSm, pLg=pLg, pRaven=pRaven)
head(d)

,pWolf,pSm,pLg,pRaven
,<dbl>,<dbl>,<dbl>,<dbl>
1,6,10,38,51
2,33,56,82,96
3,52,14,80,68
4,85,93,43,153
5,23,1,87,51
6,13,28,12,51
